In [31]:
from dotenv import load_dotenv,find_dotenv
from langchain_community.chat_models import ChatOpenAI
import pandas as pd
import os
import time  

In [32]:
load_dotenv(find_dotenv())
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
print(llm.invoke("Explain Essure"))

content="Essure is a permanent form of birth control for women that involves the insertion of small, flexible coils into the fallopian tubes. These coils cause scar tissue to form around them, blocking the tubes and preventing sperm from reaching the eggs. This method of contraception is non-hormonal and does not require surgery, as it can be done in a doctor's office without the need for anesthesia.\n\nEssure was marketed as a safe and effective alternative to traditional forms of birth control such as tubal ligation or the use of hormonal contraceptives. However, in recent years, there have been reports of serious side effects and complications associated with Essure, including chronic pain, migration of the coils, and allergic reactions. As a result, the device has been taken off the market in many countries and is no longer recommended as a form of birth control.\n\nMany women who have experienced negative side effects from Essure have filed lawsuits against the manufacturer, Bayer

In [ ]:
all_files=pd.read_csv("/Users/arama1/Desktop/all_files.csv")
#Filter out submissions that don't contain the word Essure (title or subtext) (from the allfiles df)
essure_pattern = r'\bEssure\b|\bessure\b'
essure_submissions = all_files[
        all_files['title'].str.contains(essure_pattern, regex=True, case=True, na=False) |
        all_files['selftext'].str.contains(essure_pattern, regex=True, case=True, na=False)]
essure_submissions.head(10)

essure_submissions = essure_submissions.drop_duplicates(subset=['title'])[['title', 'selftext', 'subreddit_submission', 'created_utc_submission']]


In [33]:
thirdsub=pd.read_csv("/Users/arama1/Desktop/third_test/third_sub_subset_sample_clean.csv")

In [23]:
essure_submissions['title_and_selftext'] = essure_submissions['title']+ ":" + essure_submissions['selftext']

In [35]:
thirdsub['title_and_selftext'] = thirdsub['title']+ ":" + thirdsub['selftext']
thirdsub.to_csv("/Users/arama1/Desktop/third_test/thirdsub.csv", index=False)

In [24]:
essure_submissions.to_csv("/Users/arama1/Desktop/essure_submissions.csv", index=False)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

# Initialize the tokenizer for GPT-3.5-turbo
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,  # Approximate number of tokens
    chunk_overlap=200,  # Some overlap to maintain context
    length_function=lambda text: len(encoding.encode(text)),  # Use the same token counter
    separators=["\n\n", "\n", " ", ""]  # Default separators
)
# Convert selftext to string
essure_submissions['title_and_selftext'] = essure_submissions['title_and_selftext'].astype(str)

# Apply the splitter to each text in the DataFrame
essure_submissions['chunks'] = essure_submissions['title_and_selftext'].apply(lambda x: text_splitter.split_text(x))

# Print the first few rows to verify
print(essure_submissions[['title_and_selftext', 'chunks']].head())

                                  title_and_selftext  \
0  Essure - did you know about this form of perma...   
1  Anyone have experience with Essure? I'm consid...   
2  Is there an age minimum for Essure/tubal litig...   
3  Any experiences with transcervical sterilizati...   
4  A good gynecologist story!:Ok, so, I turn 25 n...   

                                              chunks  
0  [Essure - did you know about this form of perm...  
1  [Anyone have experience with Essure? I'm consi...  
2  [Is there an age minimum for Essure/tubal liti...  
3  [Any experiences with transcervical sterilizat...  
4  [A good gynecologist story!:Ok, so, I turn 25 ...  


In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

# Initialize the tokenizer for GPT-3.5-turbo
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,  # Approximate number of tokens
    chunk_overlap=200,  # Some overlap to maintain context
    length_function=lambda text: len(encoding.encode(text)),  # Use the same token counter
    separators=["\n\n", "\n", " ", ""]  # Default separators
)
# Convert selftext to string
thirdsub['title_and_selftext'] = thirdsub['title_and_selftext'].astype(str)

# Apply the splitter to each text in the DataFrame
thirdsub['chunks'] = thirdsub['title_and_selftext'].apply(lambda x: text_splitter.split_text(x))

# Print the first few rows to verify
print(thirdsub[['title_and_selftext', 'chunks']].head())

                                  title_and_selftext  \
0  Happy happy day!:Laying in bed with my cats re...   
1  Well, this just got interesting.:I'm supposed ...   
2  NYS Sterilization Question :):Hey everyone! I'...   
3  I just came back from the doctor's office. It'...   
4  Had Essure procedure yesterday, went horseback...   

                                              chunks  
0  [Happy happy day!:Laying in bed with my cats r...  
1  [Well, this just got interesting.:I'm supposed...  
2  [NYS Sterilization Question :):Hey everyone! I...  
3  [I just came back from the doctor's office. It...  
4  [Had Essure procedure yesterday, went horsebac...  


In [26]:
# Get length of chunks list for each row
essure_submissions['chunk_count'] = essure_submissions['chunks'].apply(len)

# Show distribution of chunk counts
print("Chunk count distribution:")
print(essure_submissions['chunk_count'].value_counts().sort_index())

# Show some example rows with their chunk counts
print("\nExample rows with chunk counts:")
print(essure_submissions[['selftext', 'chunk_count']].head())

# Get summary statistics
print("\nSummary statistics of chunk counts:")
print(essure_submissions['chunk_count'].describe())

essure_submissions.info

Chunk count distribution:
chunk_count
1    1359
Name: count, dtype: int64

Example rows with chunk counts:
                                            selftext  chunk_count
0                                          [deleted]            1
1                                                nan            1
2  Some background: I'm 17, live with my somewhat...            1
3  I'm firmly decided to never have children and ...            1
4  Ok, so, I turn 25 next month. Me and my husban...            1

Summary statistics of chunk counts:
count    1359.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: chunk_count, dtype: float64


<bound method DataFrame.info of                                                    title  \
0      Essure - did you know about this form of perma...   
1      Anyone have experience with Essure? I'm consid...   
2      Is there an age minimum for Essure/tubal litig...   
3      Any experiences with transcervical sterilizati...   
4                             A good gynecologist story!   
...                                                  ...   
20993                          Help interpreting results   
20994  Should I be concerned about abnomen pain 9 mon...   
20995                        My wife is in constant pain   
20996                         Recurring mesenteric cysts   
21027  (Update) tubes tied 10 years ago, think I'm pr...   

                                                selftext subreddit_submission  \
0                                              [deleted]      TwoXChromosomes   
1                                                    nan      TwoXChromosomes   
2   

In [37]:
# Get length of chunks list for each row
thirdsub['chunk_count'] = thirdsub['chunks'].apply(len)

# Show distribution of chunk counts
print("Chunk count distribution:")
print(thirdsub['chunk_count'].value_counts().sort_index())

# Show some example rows with their chunk counts
print("\nExample rows with chunk counts:")
print(thirdsub[['selftext', 'chunk_count']].head())

# Get summary statistics
print("\nSummary statistics of chunk counts:")
print(thirdsub['chunk_count'].describe())

thirdsub.info

Chunk count distribution:
chunk_count
1    38
Name: count, dtype: int64

Example rows with chunk counts:
                                            selftext  chunk_count
0  Laying in bed with my cats resting after getti...            1
1  I'm supposed to have the Essure implant proced...            1
2  Hey everyone! I'm kind of new to this sub (not...            1
3  I'm NOT pregnant, and I never will be!! My Ess...            1
4  Modern medicine is just awesome. \nanyway, ask...            1

Summary statistics of chunk counts:
count    38.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
Name: chunk_count, dtype: float64


<bound method DataFrame.info of                                                 title  \
0                                    Happy happy day!   
1                    Well, this just got interesting.   
2                       NYS Sterilization Question :)   
3   I just came back from the doctor's office. It'...   
4   Had Essure procedure yesterday, went horseback...   
5   A question for the voluntarily sterile Ladies ...   
6   'You'll probably change your mind later' peopl...   
7   What are my options for making my period not s...   
8                                    Essure under 30?   
9   OKAY! First essure consultation is scheduled! ...   
10  About to go talk to my doctor about sterilization   
11  Tubal Ligation &amp; Novasure happening Monday...   
12                       My apologies to the CF woman   
13  Have you ever filed a complaint about an incom...   
14    DAE get advertisements for Essure on Hulu Plus?   
15  I think my doctor is trying to convince me to ...   

In [38]:
thirdsub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     38 non-null     object
 1   selftext                  38 non-null     object
 2   subreddit_submission      38 non-null     object
 3   created_utc_submission    38 non-null     object
 4   Processed_selftext        38 non-null     object
 5   Processed_title           37 non-null     object
 6   title_selftext            38 non-null     object
 7   Processed_title_selftext  38 non-null     object
 8   title_and_selftext        38 non-null     object
 9   chunks                    38 non-null     object
 10  chunk_count               38 non-null     int64 
dtypes: int64(1), object(10)
memory usage: 3.4+ KB


In [15]:
essure_submissions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1359 entries, 0 to 21027
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   1359 non-null   object
 1   selftext                1359 non-null   object
 2   subreddit_submission    1359 non-null   object
 3   created_utc_submission  1359 non-null   object
 4   chunks                  1359 non-null   object
 5   chunk_count             1359 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 74.3+ KB


In [39]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage
# Define the base prompt template
system_prompt = """You are an expert in analyzing Reddit posts for medical content."""

human_prompt = """Your task is to evaluate whether the user has had Essure, a medical device, placed in their body.:

{text}

 Direct user-experience contains the element of users who describe having the device installed in themselves.  
 Rate from 0 to 1.  0 = no experience, 1 = direct experience. 
Respond with: [score]; [brief justificaton]"""
# Create the chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt),
])


In [28]:
test_chunk = essure_submissions.iloc[1]['title_and_selftext']
print("Test chunk:")
print(test_chunk)

Test chunk:
Anyone have experience with Essure? I'm considering it.:nan


In [40]:

test_chunk = thirdsub.iloc[1]['title_and_selftext']
print("Test chunk:")
print(test_chunk)

Test chunk:
Well, this just got interesting.:I'm supposed to have the Essure implant procedure on Friday. I have the day off for my pre op tomorrow and the day of the procedure itself. Pretty cool that it's on my payday, which I have direct deposit for so this is going to be the most memorable payday ever, right?

Well, I woke up a few times last night freaking out. Totally forgot that nickel was one of the material samples in a skin allergy test panel I had done about 5 years ago through a dermatologist. So I called them the moment their office opened today... because I couldn't remember a thing from the results. 

Yep. Bullet dodged. "You just had a mild reaction though..."

I can't allow myself to have the Essure implant. A fragment of nickel taped to my back for a few days is nothing. But an implant is forever. I called the clinic I got approved through and I'm waiting to hear back from them.

I sure hope I can still get SOMETHING done on Friday, just different tools on the surgeon

In [41]:
from langchain import LLMChain
from langchain.schema.output_parser import StrOutputParser  

# Create the chain
chain = prompt_template | llm | StrOutputParser()

# Test prompt with single chunk
try:
    # Use the chain we created earlier
    response = chain.invoke({"text": test_chunk})
    
    print("Input chunk preview:")
    print(test_chunk[:300], "...\n")
    
    print("LLM Response:")
    score, justification = response.split(';', 1)
    print(f"Score: {score.strip()}")
    print(f"Justification: {justification.strip()}")
    
except Exception as e:
    print(f"Error processing text: {str(e)}")

Input chunk preview:
Well, this just got interesting.:I'm supposed to have the Essure implant procedure on Friday. I have the day off for my pre op tomorrow and the day of the procedure itself. Pretty cool that it's on my payday, which I have direct deposit for so this is going to be the most memorable payday ever, righ ...

LLM Response:
Score: 0
Justification: The user describes considering getting the Essure implant procedure but ultimately decides against it due to concerns about a nickel allergy. The user does not mention having the Essure device already placed in their body.


In [45]:
# Create the chain
# chain = prompt_template | llm | StrOutputParser()

# Process all rows in the dataframe
results = []

for index, row in thirdsub.iterrows():
    chunk_scores = []
    chunk_justifications = []
    
    for chunk_idx, chunk in enumerate(row['chunks']):
        try:
            response = chain.invoke({"text": chunk})
            
            if ';' in response:
                score_part, justification = response.split(';', 1)
                score_str = score_part.replace('Score:', '').strip()
                score = float(score_str)
                chunk_scores.append(score)
                chunk_justifications.append(justification.strip())
            
            time.sleep(1)
            
        except Exception as e:
            # Only print when there's an error
            print(f"\nError in Submission row: {row['title_and_selftext']}")
            print(f"Chunk {chunk_idx}")
            print(f"Response causing error: {response}")
            print(f"Error type: {type(e).__name__}")
            continue
    
    if chunk_scores:
        avg_score = sum(chunk_scores) / len(chunk_scores)
        results.append({
            'title_and_selftext': row['title_and_selftext'],
            'experience_score': round(avg_score, 2),
            #'num_chunks_processed': len(chunk_scores),
            'justifications': chunk_justifications
        })

In [46]:
# Convert the results list to a DataFrame
llm_experience_sub3 = pd.DataFrame(results)

# Save the DataFrame to a CSV file
llm_experience_sub3.to_csv("/Users/arama1/Desktop/Accelerator/Accelerator/code/results.csv", index=False)

# Print confirmation
print("Results saved to 'results.csv'")

Results saved to 'results.csv'


In [44]:
# Create the chain
# chain = prompt_template | llm | StrOutputParser()

# Process all rows in the dataframe
results = []

for index, row in essure_submissions.iterrows():
    chunk_scores = []
    chunk_justifications = []
    
    for chunk_idx, chunk in enumerate(row['chunks']):
        try:
            response = chain.invoke({"text": chunk})
            
            if ';' in response:
                score_part, justification = response.split(';', 1)
                score_str = score_part.replace('Score:', '').strip()
                score = float(score_str)
                chunk_scores.append(score)
                chunk_justifications.append(justification.strip())
            
            time.sleep(2)
            
        except Exception as e:
            # Only print when there's an error
            print(f"\nError in Submission row: {row['title_and_selftext']}")
            print(f"Chunk {chunk_idx}")
            print(f"Response causing error: {response}")
            print(f"Error type: {type(e).__name__}")
            continue
    
    if chunk_scores:
        avg_score = sum(chunk_scores) / len(chunk_scores)
        results.append({
            'title_and_selftext': row['title_and_selftext'],
            'experience_score': round(avg_score, 2),
            #'num_chunks_processed': len(chunk_scores),
            'justifications': chunk_justifications
        })


Error in Submission row: I just need to get this off my chest:I'm almost 32, been in a great relationship with a guy for about a year and a half. prior to that I was single and having a lot of single fun for about 6 months( which led me to get genital warts, but that's a story for another day, sigh) and prior to THAT, I was with a guy/married for 13 years. That guy was my high school boyfriend. A whole lot of mental illness on his part, infidelity on both parts, and mental mindfuckery from both of us led to the breakdown of that marriage. In 2006, at age 25, I got Essure sterilization, because of a prior abortion and a prior miscarriage, at the time I felt like it was the best thing to do, because I was not in a state at that time to have children, ex didnt want them, I thought I didnt either. Now since my 30s have hit, baby rabies is in full effect. Every goddamn month when I'm ovulating, it's at it's worst, but honestly not a day goes by where I don't mourn the loss of my fertility,

KeyboardInterrupt: 